In [12]:
print ("============================================================")
print (" Course Work                              Robiul Islam      ")
print ("============================================================")


import pandas as pd 
from sklearn.utils import shuffle

data = pd.read_csv("datafull.csv",sep=';') 
# Remove Null value
data.dropna(inplace = True)
data = shuffle(data, random_state=7)
data.head()

 Course Work                              Robiul Islam      


,work,separation,partner,marriage,children
283,18.0,20.0,20.0,21.0,22.0
1329,17.0,25.0,20.0,20.0,29.0
2051,25.0,25.0,25.0,34.0,35.0
450,17.0,19.0,30.0,24.0,26.0
1135,19.0,19.0,23.0,24.0,25.0


In [15]:
for i in data:
    print(i)

work
separation
partner
marriage
children


In [23]:
D = pd.read_table('/Users/HSE/coursework/python_vmv/SeqDem/result.txt',header=None)

for i in D:
    print(i)

0
